In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("mysportsfeeds_apikey")

def snaps_api(api_key, season, week):
    api_url = f"https://api.mysportsfeeds.com/v2.1/pull/nfl/{season[0]}/week/{week[0]}/player_gamelogs.json"

    try:
        response = requests.get(
            url=f"{api_url}",
            params={
                "fordate": "20211103"
            },
            headers={
                "Authorization": "Basic " + base64.b64encode("{}:{}".format(api_key, "MYSPORTSFEEDS").encode("utf-8")).decode("ascii")
            }
        )
        print("Response HTTP Status Code: {status_code}".format(
            status_code=response.status_code))
    except requests.exceptions.RequestException:
        print("HTTP Request failed")

    r = response.content
    soup = BeautifulSoup(r, "html.parser")
    json = soup.text
    df = pd.json_normalize(json["gamelogs"])
    cols = list(df.columns)
    not_drop = []
    for c in cols:
        if "game." in c or "player." in c or "team." in c or "snapCounts" in c:
            not_drop.append(c)
            
    snaps = df[not_drop]

    time.sleep(5)
    
    return snaps

def all_snaps(seasons, weeks):
    start = time.time()
    final_df = pd.DataFrame()
    for s in seasons:
        for w in weeks:
            snaps = snaps_api(s, w)
            final_df = pd.concat([snaps, final_df])
            
    print(time.time()-start)
    return final_df

In [ ]:
seasons = ["2020-2021-regular", "2019-2020-regular", "2018-2019-regular"]
weeks = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17"]

In [ ]:
all_snaps = all_snaps(seasons, weeks)

In [ ]:
all_snaps["stats.snapCounts.offenseSnaps"].unique()

In [ ]:
all_snaps["stats.snapCounts.defenseSnaps"].unique()

In [ ]:
all_snaps["stats.snapCounts.specialTeamSnaps"].unique()